In [ ]:
# You will need to run this block twice to make it effective
!apt-get update 
!apt-get install cmake 
!pip install --upgrade setuptools 
!pip install ez_setup 
!pip install gym==0.24.1
!pip install gym[all]

!pip install gym pyvirtualdisplay 
!apt-get install -y xvfb python-opengl ffmpeg 

In [ ]:
from importlib import reload
import utils
reload(utils)
from utils import*

import gym
from gym.wrappers.monitoring import video_recorder
from gym.wrappers.monitoring.video_recorder import VideoRecorder
from gym.wrappers.record_video import RecordVideo
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

import torch
import torch.nn as nn
import numpy as np
from torch import nn
import copy
from collections import deque
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import math

from torch import randint
from time import sleep
import pickle
import statistics as st
from gym.core import RewardWrapper
import gc



display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""


def simulate(agent=None,env=None,epsilon=0,memory=None,render=False):
  agent.eval()
  env.reset()
  if(render):
    env.env = RecordVideo(env.env, './video')
    env.env.render()
  state,rew,done,info = env.skip_episodes(70,[0,0.1,0])
  ep_len = 0
  while not done:
      # exploitation(0) vs exploration(1)
      sample = torch.bernoulli(torch.tensor(epsilon).float())
      if(sample==1):
        A = torch.randint(0,3,(1,))
      else:
        A = agent.get_action(state)

      # progress a time step
      next_state, rew, done, info = env.step(agent.convert_action(A,state))
      #plot_image(state)
      #print(rew)

      if(done):
        break

      # collect memory
      if(memory!=None):
        memory.collect([state, A, rew, next_state])
      state = next_state

      ep_len = env.ep_len
      # stop criteria 
      if(ep_len>2000):
        break

  # readd 100 to episode reward to resync measured reward with documentation (undo the -100 penalty)
  score = env.real_rew
  if(render):
    print("score",score,"ep_len",ep_len)
    env.env.close()
    show_video()
  
  return score,ep_len


def test_model(agent, env, episodes=1):
  rewards = []
  ep_lens = []
  for i in range(0,episodes):
    rew,ep_len = simulate(agent,env)
    rewards.append(rew)
    ep_lens.append(ep_len)
    print("Test "+str(i+1)+"/"+str(episodes)+": reward =",rew," episode len =",ep_len)
  print("\nAverage Reward = ",sum(rewards)/len(rewards),"Average Ep_len = ",sum(ep_lens)/len(ep_lens),"\n")
  return rewards,ep_lens


class ExperienceReplay(object):
# one entry is [state,action,reward,next_state]
  def  __init__(self, length):
    self.experience_replay = deque(maxlen=length)
  def collect(self,experience):
    self.experience_replay.append(experience)
    return
  def sample_from_experience(self, sample_size):
    sample_size = min(sample_size,len(self.experience_replay))
    sample = random.sample(self.experience_replay,sample_size)
    state = torch.tensor([episode[0] for episode in sample]).float()
    action = torch.tensor([episode[1] for episode in sample]).float()
    reward = torch.tensor([episode[2] for episode in sample]).float()
    next_state = torch.tensor([episode[3] for episode in sample]).float()

    return state,action,reward,next_state



class DQN_Network_Simple(nn.Module):
  def __init__(self,gamma = None):
    super().__init__()
    #layers
    self.LeakyReLU = nn.LeakyReLU()
    self.conv1 = nn.Conv2d(1,4,kernel_size = 7, stride = 4,padding = 0)
    self.conv2 = nn.Conv2d(4,8,kernel_size = 3, stride = 1,padding = 2)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1 = nn.Linear(289,100)
    self.fc2 = nn.Linear(100,3)
    self.batchnormCNN1 = nn.BatchNorm2d(num_features = 4)
    self.batchnormCNN2 = nn.BatchNorm2d(num_features = 8)
    self.flatten = nn.Flatten()
    self.gamma = gamma
  def forward(self,x):
    # reformat image (input = BS,96,96, or 96,96) (output = BS,1,96,96)
    x = torch.from_numpy(np.ascontiguousarray(x)).float()
    if(x.dim()==2):
      x = torch.unsqueeze(x,dim=0)
      x = torch.unsqueeze(x,dim=0)
    elif(x.dim()==3):
      x = torch.unsqueeze(x,dim=1)
    subimage = (x[:,:,84:96,13:14]-0.495)*10
    speed = torch.sum(subimage,dim=(2,3))
    x = x[:,:,:84,:]
    #plot_image(np.squeeze(x.detach().numpy()))
    
    #print(x.shape)
    #x = self.batchnormCNN1(self.LeakyReLU(self.conv1(x)))
    x = self.LeakyReLU(self.conv1(x))
    #print(x.shape)
    x = self.pool(x)
    #print(x.shape)
    #x = self.batchnormCNN2(self.LeakyReLU(self.conv2(x)))
    x = self.LeakyReLU(self.conv2(x))
    #print(x.shape)
    x = self.pool(x)
    #print(x.shape)
    x = self.flatten(x)
    #print(x.shape)
    x = torch.cat((x,speed),dim=1)
    #x = self.batchnormFC1(self.LeakyReLU(self.fc1(x)))
    x = self.LeakyReLU(self.fc1(x))
    #print(x.shape)
    x = self.fc2(x)
    #print(x.shape)
    return x
  def get_action(self,state):
    qvals = self.forward(state)
    return torch.argmax(qvals,1) 
  def convert_action(self,action,state):
    # determine if you are going too fast
    speed = get_speed(state).item()
    if(speed>3.5):
      accel = 0
    elif(speed>2.5):
      accel = 0
    else:
      accel = 0.1
    # convert action from index, to a list of turning,engine,breaking strengths
    action = action.item()
    # Discretized action space (left-forward,straight-forward,right-forward)
    if(action == 0):
      return [-0.3,accel,0]
    elif(action == 1):
      return [0,accel,0]
    elif(action == 2):
      return [0.3,accel,0]


### Training

In [ ]:
def load_memory(new,epsilon,exp_replay_size,initial_size=None):
  if(initial_size==None):
    initial_size=exp_replay_size
  # Create the model
  env = wrap_env(gym.make("CarRacing-v1").unwrapped)
  agent = DQN_Network_Simple()
  if(not new):
    agent.load_state_dict(torch.load("car-racing-dqn.pth"))
  memory = ExperienceReplay(exp_replay_size)

  # initiliaze experience replay
  index = 0
  for i in range(exp_replay_size):
      state = env.reset()
      simulate(agent,env,epsilon = epsilon, memory = memory)
      if(len(memory.experience_replay)>=initial_size):
        break
      print(len(memory.experience_replay))

  return memory


def update(agent,optimizer,loss_func,target_agent,memory,batch_size):
  agent.train()
  target_agent.eval()
  # current (S,A) Qval
  state,action,reward,next_state = memory.sample_from_experience(batch_size)
  Qvals = agent(state)
  curr_Qval = Qvals[torch.arange(Qvals.size(0)),action.long()]
  
  # best next (S,A) Qval
  with torch.no_grad():
    next_Qval, indices = torch.max(target_agent(next_state),dim=1)

  # update agent
  #print(reward + agent.gamma*next_Qval,curr_Qval)
  loss = loss_func(reward + agent.gamma*next_Qval, curr_Qval)
  loss.backward(retain_graph = False)
  optimizer.step()
  optimizer.zero_grad()


def train(new,num_ep,lr_start,epsilon_start,gamma,memory):
  # set hyperparamters
  agent = DQN_Network_Simple(gamma=gamma)
 
  # start new run
  if(new):
    reward_hist = []; ep_len_hist = []; lr_hist = []; epsilon_hist = []
  # load previous runs
  else:
    agent.load_state_dict(torch.load("car-racing-dqn.pth")); reward_hist = load_list("reward_hist.data");ep_len_hist = load_list("ep_len_hist.data");epsilon_hist = load_list("epsilon_hist.data");lr_hist = load_list("lr_hist.data")

  #initialize models
  target_agent = DQN_Network_Simple(agent.gamma)
  target_agent.load_state_dict(agent.state_dict())
  env = wrap_env(gym.make("CarRacing-v1").unwrapped)
  optimizer = torch.optim.SGD(agent.parameters(),lr_start)
  MSELoss = torch.nn.MSELoss()

  # training loop
  for ep_num in tqdm(range(0,num_ep)):
    lr = lr_start*(0.99042**ep_num)
    epsilon = epsilon_start*(0.99424**ep_num)

    for param_group in optimizer.param_groups:
      param_group['lr'] = lr

    state, done, losses, ep_len, reward = env.reset(), False, 0, 0, 0
    reward,ep_len = simulate(agent,env,epsilon = epsilon, memory = memory)       
  
    for i in range(0,30):
      update(agent,optimizer,MSELoss,target_agent,memory,batch_size=32)
    target_agent.load_state_dict(agent.state_dict())
    gc.collect(generation=2)

    if(ep_num%3==0):
      reward, ep_len = test_model(agent=agent,env=env, episodes=1)
      print("Settings: lr =",lr,"epsilon =",epsilon)
      print("Test Result: reward =",reward[0],"episode length =",ep_len[0])
      reward_hist.append(reward[0])
      ep_len_hist.append(ep_len[0])
      lr_hist.append(lr)
      epsilon_hist.append(epsilon)
    
    if(ep_num%30==0):
      # save results
      torch.save(agent.state_dict(),"car-racing-dqn.pth");save_list(reward_hist,"reward_hist.data");save_list(ep_len_hist,"ep_len_hist.data");save_list(epsilon_hist,"epsilon_hist.data");save_list(lr_hist,"lr_hist.data")

In [ ]:
memory = load_memory(new=True,epsilon=1,exp_replay_size=500)
train(new=True,num_ep = 360,lr_start=0.0003,epsilon_start=0.8,gamma=0.92,memory=memory)

/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:98: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


12
20
43
62
71
103
111
134
148
159
167
176
211
220
233
241
261
272
280
310
320
328
340
351
364
372
384
391
405
414
433
445
457
473
490


  0%|          | 1/360 [00:05<33:38,  5.62s/it]

Test 1/1: reward = 27.494444444444394  episode len = 105

Average Reward =  27.494444444444394 Average Ep_len =  105.0 

Settings: lr = 0.0003 epsilon = 0.8
Test Result: reward = 27.494444444444394 episode length = 105


  1%|          | 4/360 [00:19<30:02,  5.06s/it]

...

Test 1/1: reward = 207.1000000000015  episode len = 425

Average Reward =  207.1000000000015 Average Ep_len =  425.0 

Settings: lr = 2.2302816213769293e-05 epsilon = 0.16815992779658168
Test Result: reward = 207.1000000000015 episode length = 425


 76%|███████▌  | 274/360 [35:42<12:57,  9.05s/it]

Test 1/1: reward = 146.77407407407478  episode len = 270

Average Reward =  146.77407407407478 Average Ep_len =  270.0 

Settings: lr = 2.166795428329594e-05 epsilon = 0.1652708295368555
Test Result: reward = 146.77407407407478 episode length = 270


 77%|███████▋  | 276/360 [35:55<10:56,  7.81s/it]


KeyboardInterrupt: ignored

In [ ]:
env = wrap_env(gym.make("CarRacing-v1").unwrapped)
agent = DQN_Network_Simple()
agent.load_state_dict(torch.load("car-racing-dqn.pth"))

simulate(agent=agent,env=env,render=True)
#test_model(agent,env,episodes=10)

/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:98: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "
/usr/local/lib/python3.7/dist-packages/gym/wrappers/record_video.py:76: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  f"Overwriting existing videos at {self.video_folder} folder "


score 453.7528052805233 ep_len 740


(453.7528052805233, 740)

In [ ]:
print("lr_hist")
hist = load_list("lr_hist.data")
for stage in hist:
  print(stage)
print("\nepsilon_hist")
hist = load_list("epsilon_hist.data")
for stage in hist:
  print(stage)
print("\nep_len_hist")
hist = load_list("ep_len_hist.data")
for stage in hist:
  print(st.mean(stage))
print("\nreward_hist")
hist = load_list("reward_hist.data")
for stage in hist:
  print(st.mean(stage))